In [1]:
import open3d as o3d
import numpy as np
import os
from open3d.web_visualizer import draw

from utils.depth_utils import *
from utils.register_utils import *

output_dir = "output_archived_1"

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D INFO] Resetting default logger to print to terminal.


In [15]:
# Register the mesh to the original point cloud
print("[INFO] Registering mesh to original point cloud...")

# object_id = args.obj_id
object_id = "05117"\

np.random.seed(4)

# Load the ground truth mesh
complete_pcd = o3d.io.read_triangle_mesh(f"./redwood_dataset/GT/{object_id}.ply")
complete_pcd = complete_pcd.sample_points_uniformly(number_of_points=16384)
complete_pcd.points = o3d.utility.Vector3dVector(-np.asarray(complete_pcd.points))
translate = -complete_pcd.get_center()
complete_pcd = complete_pcd.translate(translate)
scale = 0.5 / np.max(np.linalg.norm(np.asarray(complete_pcd.points), axis=1))
complete_pcd = complete_pcd.scale(scale, center=complete_pcd.get_center())

# Load the partial point cloud
partial_pcd = o3d.io.read_point_cloud(f"./redwood_dataset/point_clouds/{object_id}.ply")
partial_pcd.points = o3d.utility.Vector3dVector(-np.asarray(partial_pcd.points))
partial_pcd = partial_pcd.translate(translate)
partial_pcd = partial_pcd.scale(scale, center=complete_pcd.get_center())
partial_pcd = partial_pcd.farthest_point_down_sample(num_samples=16384)

# Load the generated mesh
mesh = o3d.io.read_triangle_mesh(os.path.join(output_dir, f"{object_id}_mesh.ply"))
# Normalize the mesh
mesh = mesh.translate(-mesh.get_center())
mesh = mesh.scale(0.5 / np.max(np.linalg.norm(np.asarray(mesh.vertices), axis=1)), center=mesh.get_center())
mesh_pcd = mesh.sample_points_uniformly(number_of_points=16384)
mesh_pcd.points = o3d.utility.Vector3dVector(np.asarray(mesh_pcd.points) * np.array([-1, 1, 1]))

# Visualize the partial point cloud and the mesh
mesh_pcd.paint_uniform_color([0,0,1])
partial_pcd.paint_uniform_color([1, 0, 0])
complete_pcd.paint_uniform_color([0, 1, 0])

# Draw the partial point cloud and the mesh
draw([partial_pcd, mesh_pcd, complete_pcd])

[INFO] Registering mesh to original point cloud...
[Open3D INFO] Window window_7 created.


WebVisualizer(window_uid='window_7')

In [16]:
# Register the mesh to the original point cloud
scales = np.arange(0.8, 1.25, 0.025)
# scales = np.meshgrid(scales, scales, scales)
# scales = np.array(scales).reshape(3, -1).T
# print(f"Number of scales: {scales[..., None].shape}")
transformation = multiscale_registration(mesh_pcd, partial_pcd, scales, voxel_size=0.005)

mesh_pcd.transform(transformation)

# Compute the chamfer distance
cd_o3d = (np.mean(complete_pcd.compute_point_cloud_distance(mesh_pcd)) + \
    np.mean(mesh_pcd.compute_point_cloud_distance(complete_pcd))) / 2
print(f"Chamfer Distance: {cd_o3d}")

# from utils.chamfer_python import distChamfer
# cd_left, cd_right, index_left, index_right = distChamfer(torch.tensor(np.asarray(complete_pcd.points)).unsqueeze(0), 
#                  torch.tensor(np.asarray(mesh_pcd.points)).unsqueeze(0))
# print(cd_left.shape, cd_right.shape)
# cd = (cd_left.sqrt().mean(dim=1) + cd_right.sqrt().mean(dim=1))
# print(f"Chamfer Distance (PyTorch): {cd.item()}")

[Open3D WARNING] Too few correspondences (1420) after mutual filter, fall back to original correspondences.
[Open3D WARNING] Too few correspondences (1418) after mutual filter, fall back to original correspondences.
[Open3D WARNING] Too few correspondences (1418) after mutual filter, fall back to original correspondences.
[Open3D WARNING] Too few correspondences (1418) after mutual filter, fall back to original correspondences.
[Open3D WARNING] Too few correspondences (1420) after mutual filter, fall back to original correspondences.
[Open3D WARNING] Too few correspondences (1420) after mutual filter, fall back to original correspondences.
[Open3D WARNING] Too few correspondences (1420) after mutual filter, fall back to original correspondences.
[Open3D WARNING] Too few correspondences (1418) after mutual filter, fall back to original correspondences.
[Open3D WARNING] Too few correspondences (1419) after mutual filter, fall back to original correspondences.
[Open3D WARNING] Too few cor

In [17]:

draw([mesh_pcd, partial_pcd, complete_pcd])

[Open3D INFO] Window window_8 created.


WebVisualizer(window_uid='window_8')